In [30]:
import glob
import pandas as pd
import numpy as np
import os
import shutil
import math
import pickle
import random

import contextlib
import ipywidgets as widgets
from ipywidgets import VBox
import io
from PIL import Image
import matplotlib.pyplot as plt

In [31]:
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.layers import *
from funciones import *
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
df_nodulos=pd.read_csv("nodulos_descripcion110.csv",header=0,index_col=0)
df_nodulos.columns=["Forma","Margen","Orientación","Ecogenicidad","Característica Posterior","Halo Ecogénico", 'Sugestivo', 'Forma2',
       'Margen2', 'Orientación2', 'Ecogenicidad2', 'Característica Posterior2',
       'Halo Ecogénico2', 'Sugestivo2', 'BIRADS', 'Resultados', 'Num Nódulos']
df_nodulos["Ecogenicidad"].replace({"compleja": "heterogénea", "mixta":"heterogénea"}, inplace=True)
df_nodulos["Ecogenicidad2"].replace({"compleja": "heterogénea", "mixta":"heterogénea"}, inplace=True)


Formas=[l for l in list(set(list(df_nodulos["Forma"])+list(df_nodulos["Forma2"]))) if l==l]
Margenes=[l for l in list(set(list(df_nodulos["Margen"])+list(df_nodulos["Margen2"]))) if l==l]
Orientaciones=["paralela","antiparalela"]
Ecogenicidades=[l for l in list(set(list(df_nodulos["Ecogenicidad"])+list(df_nodulos["Ecogenicidad2"]))) if l==l]
Ecogenicidades.remove("hiperecoica")
Posteriores=["sin cambios","refuerzo","sombra","mixto"]
Posteriores.remove("mixto")
Halos=["sí","no"]
Sugestividades=[l for l in list(set(list(df_nodulos["Sugestivo"])+list(df_nodulos["Sugestivo2"]))) if l==l]
Sugestividades2=[l for l in list(set(list(df_nodulos["Sugestivo"])+list(df_nodulos["Sugestivo2"]))) if l==l]
Sugestividades2.append("otro")
Sugestividades.remove("ganglio intramamario")
Sugestividades.remove("fibroadenoma con degeneración quística")
Sugestividades.remove("quiste tóxico")
Sugestividades.remove("lesión papilar intraquística")
Sugestividades2.remove("ganglio intramamario")
Sugestividades2.remove("fibroadenoma con degeneración quística")
Sugestividades2.remove("quiste tóxico")
Sugestividades2.remove("lesión papilar intraquística")
#BIRADS=["2","3","4A","4B","4C","5"]
Resultados=["benign","malignant"]
#Tipo=["quiste","fibroadenoma","carcinoma ductal invasivo","carcinoma ductal in situ","linfoma"]
#voc=set(Formas+Margenes+Orientaciones+Ecogenicidades+Posteriores+Halos+Sugestividades+BIRADS+Resultados)

voc2=set(Formas+Margenes+Orientaciones+Ecogenicidades+Posteriores+Halos+Sugestividades2+Resultados)
voc2=list(voc2)
voc2.sort()

Formas.sort()
Margenes.sort()
Ecogenicidades.sort()
Orientaciones.sort()
Posteriores.sort()
Halos.sort()
Sugestividades.sort()
Sugestividades2.sort()
Resultados.sort()

BIRADS=["2","3","4A","4B","4C","5"]
BIRADS.sort()

FileNotFoundError: [Errno 2] No such file or directory: 'nodulos_descripcion110.csv'

In [22]:
word_to_idx_formas = dict((word, idx) for idx, word in enumerate(Formas))
idx_to_word_formas = dict((idx, word) for idx, word in enumerate(Formas))

word_to_idx_margenes = dict((word, idx) for idx, word in enumerate(Margenes))
idx_to_word_margenes = dict((idx, word) for idx, word in enumerate(Margenes))

word_to_idx_orientaciones = dict((word, idx) for idx, word in enumerate(Orientaciones))
idx_to_word_orientaciones = dict((idx, word) for idx, word in enumerate(Orientaciones))

word_to_idx_ecogenicidades = dict((word, idx) for idx, word in enumerate(Ecogenicidades))
idx_to_word_ecogenicidades = dict((idx, word) for idx, word in enumerate(Ecogenicidades))

word_to_idx_posteriores = dict((word, idx) for idx, word in enumerate(Posteriores))
idx_to_word_posteriores = dict((idx, word) for idx, word in enumerate(Posteriores))

word_to_idx_halos = dict((word, idx) for idx, word in enumerate(Halos))
idx_to_word_halos = dict((idx, word) for idx, word in enumerate(Halos))

word_to_idx_sugestividades = dict((word, idx) for idx, word in enumerate(Sugestividades))
idx_to_word_sugestividades = dict((idx, word) for idx, word in enumerate(Sugestividades))

word_to_idx_sugestividades2 = dict((word, idx) for idx, word in enumerate(Sugestividades2))
idx_to_word_sugestividades2 = dict((idx, word) for idx, word in enumerate(Sugestividades2))

word_to_idx_resultados = dict((word, idx) for idx, word in enumerate(Resultados))
idx_to_word_resultados = dict((idx, word) for idx, word in enumerate(Resultados))

word_to_idx_birads = dict((word, idx) for idx, word in enumerate(BIRADS))
idx_to_word_birads = dict((idx, word) for idx, word in enumerate(BIRADS))

In [23]:
class Horizontal_flip(Layer):
    def __init__(self,**kwargs):
        super(Horizontal_flip, self).__init__()
        super(Horizontal_flip, self).__init__(**kwargs)
    def call(self,inputs,training=None):
        if not training:
            output=inputs
        else:
            output=tf.image.random_flip_left_right(inputs)
        return output
  

In [24]:
class Attention(Layer):
    def __init__(self, dropout=False, L2Attention=False, Gatted=False, L2dim=20,name="Attention",**kwargs):
        #Inicializa los features que no dependen de la entrada
        self.L2Attention=L2Attention
        self.Gatted=Gatted
        self.L2dim=L2dim
        self.weight_initializer = tf.keras.initializers.glorot_normal
        self.const_initializer = tf.keras.initializers.Zeros()
        self.dropout=dropout
        

        super(Attention, self).__init__(**kwargs)
        
    def build(self,input_shape):
        Features_shape=input_shape
        F_1=Features_shape[1]
        F_2=Features_shape[2]
        #Inicia los features que dependen de la entrada
        
        #Iniciar pesos  Atención
        if self.L2Attention or self.Gatted:
            self.w_a_1= self.add_weight("w_a_1", shape=[F_2,self.L2dim])
            self.b_a_1=self.add_weight("b_a_1",shape=[self.L2dim])
            
            self.w_a_2= self.add_weight("w_a_2", shape=[self.L2dim,1])
            self.b_a_2=self.add_weight("b_a_2",shape=[1])
            
            if self.Gatted:
                self.w_a_g= self.add_weight("w_a_g", shape=[F_2,self.L2dim])
                self.b_a_g=self.add_weight("b_a_g",shape=[self.L2dim])
             
        else:
            self.w_a= self.add_weight("w_a", shape=[F_2,1])
            self.b_a=self.add_weight("b_a",shape=[1])
        

    def call(self, inputs, training=None):
        #Crear primer estado oculto y memoria
        input_data=inputs

        #Primear atención
        #lstm
        if self.L2Attention or self.Gatted:
            Attention1=tf.nn.tanh(reshape_matmul(input_data,self.w_a_1)+self.b_a_1) #(B,F_1,L2dim)

            if self.Gatted:
                Attention2=k.activations.sigmoid(reshape_matmul(input_data,self.w_a_g)+self.b_a_g) #(B,F_1,L2dim)
                Attention1=tf.math.multiply(Attention1,Attention2)
            Attention=reshape_matmul(Attention1,self.w_a_2)+self.b_a_2 #(B,F_1,1)
        else:
            Attention=tf.nn.tanh(reshape_matmul(input_data,self.w_a)+self.b_a) #(B,F_1,1)

        if self.dropout and False:
            alfa=tf.nn.softmax(tf.nn.dropout(Attention[:,:,0],0.5))
        else:
            alfa=tf.nn.softmax(Attention[:,:,0]) #(B,F_1)
        context = tf.reduce_sum(tf.math.multiply(input_data, tf.expand_dims(alfa, axis=2)), 1) #(B,F_2)
        if self.dropout and training:
            context=tf.nn.dropout(context,0.5)
                                                                                        
        return context, alfa
    
    def get_config(self):
        config = super(Attention, self).get_config()
        return config

In [25]:
Encoder = k.models.load_model('EncoderYOLOSAtt2.h5',custom_objects={"Horizontal_flip":Horizontal_flip})
Dense_model=k.models.load_model("DecoderDenseYOLOSAtt2.h5")
Attention_layer=Attention()


inputs=Encoder.inputs
features=Encoder(inputs)
context,alfa=Attention_layer(features)
outputs=Dense_model(context)

Inference_model=k.Model(inputs=inputs,outputs=outputs)
print(Inference_model.summary())

with open("YOLO_SAtt_weights_3Channel22.pkl", "rb") as open_file:
    LSTM_list = pickle.load(open_file)

    
Attention_layer.set_weights(LSTM_list)

OSError: SavedModel file does not exist at: EncoderYOLOSAtt2.h5\{saved_model.pbtxt|saved_model.pb}

In [26]:
Birads_simple=k.models.load_model("MLP_simple2.h5")

OSError: SavedModel file does not exist at: MLP_simple2.h5\{saved_model.pbtxt|saved_model.pb}

In [27]:
def generate_caption(words,enc_m,dec_m):
    len_frase=len(words)
    boolean=False
    boolean2=False
    i_sug=0
    caption=""
    for i,word in enumerate(words):
        frase=""
        if word=="/n":
            continue
        elif word in Formas:
            if word!="irregular":
                frase=word[:-1]+"o"
            else:
                frase+=word
        elif word in Margenes:
            frase=word
        elif word in Orientaciones:
            frase="con orientación "+word
        elif word in Ecogenicidades:
            if word=="compleja":
                frase="con ecogenicidad compleja"
            elif word=="mixta":
                frase="con ecogenicidad mixta"
            else:
                frase=word[:-1]+"o"
        elif word in Posteriores:
            if word=="mixto":
                frase="con característica posterior mixta"
            elif word=="sin cambios":
                frase="sin cambios posteriores"
            else:
                frase="con "+word+ " posterior"
        elif word in Halos:
            if word=="no":
                frase="sin halo ecogénico"
            elif word=="sí":
                frase="con halo ecogénico"
        elif word in Sugestividades:
            frase="sugestivo de "+ word
            boolean=True
            i_sug=i
        elif word in BIRADS:
            if boolean:
                frase=" (BIRADS® "+word+")"
            else:
                i_sug=i
                if word=="2":
                    frase=" (BIRADS® "+word+")"
                if word=="3":
                    frase="probablemente benigno "+"(BIRADS® "+word+")"
                elif word=="4A":
                    frase="de baja sospecha "+"(BIRADS® "+word+")"
                elif word=="4B":
                    frase="de sospecha intermedia "+"(BIRADS® "+word+")"
                elif word=="4C":
                    frase="de alta sospecha "+"(BIRADS® "+word+")"
                elif word=="5":
                    frase="sugestivo de carcinoma "+"(BIRADS® "+word+")"
        else:
            if not boolean2:
                frase=". Resultado final aprendido por base de datos: "
                boolean2=True
            if word in Resultados:
                if word=="benign":
                    frase+="benigno"
                else:
                    frase+="maligno"
#            elif word in Tipo:
#                frase+=word
        if frase!="":
            if i==0:
                caption+="Nódulo "+frase
            elif word in Ecogenicidades:
                if word[:2]=="hi" or word[0]=="i":
                    caption+=" e "+frase
                else:
                    caption+=" y "+frase
            elif i==0:
                caption+="Nódulo "+frase
            elif frase[0]=="." or frase[:2]==" (":
                caption+=frase
            elif caption[-2:]==": ":
                caption+=frase
            elif i==len(frase)-1:
                caption+=", "+frase+"."
            else:
                caption+=", " +frase
    return caption

In [28]:
def results_simple(imag,model):
    result=model(np.reshape(imag,(1,450,450,1)))
    result=[result[1],result[3],result[4],result[0],result[5],result[2],result[7],result[6]]
    output_ejemplo=[]
    t=0
    oval_bool=False
    redondeada_bool=False
    threshold=0
    total=[]
    for carac in result:
        maxi=np.max(carac)
        mask=maxi>threshold
        result_carac=np.argmax(carac)+1
        final=result_carac*mask
        if t==0:
            m=np.zeros(len(Formas))
            if final!=0:
                m[final-1]=1
                word=idx_to_word_formas[final-1]
                output_ejemplo.append(word)
                if word=="ovalada":
                    oval_bool=True
                elif word=="redondeada":
                    redondeada_bool=True
            total.append(m)
        if t==1:
            m=np.zeros(len(Margenes))
            if final!=0:
                m[final-1]=1
                word=idx_to_word_margenes[final-1]
                output_ejemplo.append(word)
            total.append(m)
        if t==2:
            m=np.zeros(len(Orientaciones))
            if maxi>0.45:
                m[final-1]=1
                if oval_bool:
                    output_ejemplo.append("paralela")
                elif redondeada_bool:
                    t=t+1
                    continue
                else:
                    word=idx_to_word_orientaciones[final-1]
                    output_ejemplo.append(word)
            total.append(m)
        if t==3:
            m=np.zeros(len(Ecogenicidades))
            if final!=0:
                m[final-1]=1
                word=idx_to_word_ecogenicidades[final-1]
                output_ejemplo.append(word)
            total.append(m)
        if t==4:
            m=np.zeros(len(Posteriores))
            if maxi>0.3:
                m[final-1]=1
                word=idx_to_word_posteriores[final-1]
                output_ejemplo.append(word)
            total.append(m)
        if t==5:
            m=np.zeros(len(Halos))
            if final!=0:
                m[final-1]=1
                word=idx_to_word_halos[final-1]
                output_ejemplo.append(word)
            total.append(m)
        if t==6:
            m=np.zeros(len(Sugestividades2))
            if maxi>0:
                m[final-1]=1
                word=idx_to_word_sugestividades2[final-1]
                if word!="otro":
                    output_ejemplo.append(word)
            total.append(m)
        if t==7:
            m=np.zeros(len(Resultados))
            if final!=0:
                word=idx_to_word_resultados[final-1]
                output_ejemplo.append(word)
            total.append(m)
        t=t+1
    # output_ejemplo=[output_ejemplo[1],output_ejemplo[3],output_ejemplo[0],
    #                     output_ejemplo[4],output_ejemplo[5],output_ejemplo[2],output_ejemplo[7],output_ejemplo[6]]

    total=np.concatenate(total)
    if "espiculado" in output_ejemplo:
        birads="5"
    if "quiste simple" in output_ejemplo:
        birads="2"
    elif "quiste complejo" in output_ejemplo:
        birads="4A"
    # elif "fibroadenoma" in desc:
    #     BIRADS_out[key]="3"
    else:
        birads=idx_to_word_birads[np.argmax(Birads_simple(total.reshape((1,len(voc2)))))]
    output_ejemplo=output_ejemplo[:-1]+[birads]+[output_ejemplo[-1]]
    frase=generate_caption(output_ejemplo,1,2)
    
    return(frase)

In [29]:
tamaño=450
def results(imagen,imagen_dir="..\YOLO_para_describir\images"):
    imag_dir=os.path.join(imagen_dir,imagen)
    
    if not glob.glob(os.path.join("runs/detect/Yolo_test_lstm",imagen)):
        !python detect.py --weights YOLO_total.pt --conf 0.25 --hide-labels --source {imag_dir} --data data/YOLO.yaml --exist-ok --name Yolo_test_lstm --save-crop
    original=load_img(os.path.join("runs/detect/Yolo_test_lstm",imagen),color_mode="grayscale")
    
    nodulos=glob.glob(os.path.join("runs/detect/Yolo_test_lstm/crops/nodulo",imagen[:-4]+"_*"))
    print("Número de nódulos: ",len(nodulos))
    plt.imshow(original,cmap="gray")
    plt.axis('off')
    plt.show()
    i=1
    for ima in nodulos:
        if i==1:
            imag=load_img(ima,color_mode="grayscale")
        else:
            imag=load_img(ima,color_mode="grayscale")
        ancho,largo=imag.size
        
        plt.imshow(imag,cmap="gray")
        plt.axis('off')
        plt.show()
        if ancho>450 or largo>450:
            ratio=ancho/largo
            if ratio>1:
                imag=imag.resize((450,int(450/ratio)))

            else:
                imag=imag.resize((int(450*ratio),450))
            ancho,largo=imag.size
        imag=img_to_array(imag)/255.
        imag=tf.image.pad_to_bounding_box(
        imag, (tamaño-largo)//2,(tamaño-ancho)//2, 450, 450
        )
        imag=img_to_array(imag)
        words=results_simple(imag,Inference_model)
        
        print("DESCRIPCIÓN:\n",words+".")
        #print("Oval, circumscribed, isoechoic mass, with parallel orientation and no echogenic halo.\nSuggestive of fibroadenoma. BIRADS® 3. Result predicted from dataset labels: benign.")
#         print("Irregular, spiculated, hipoechoic mass, with posterior shadowing, antiparallel orientation and echogenic halo.\nBIRADS® 5. Result predicted from dataset labels: malignant.")
        #visualize(imagen=imag,alfas=np.array(alfas[:,0]))
        i=i+1
    if not nodulos:
        print("Sin hallazgos. (BIRADS® 1)")

In [14]:
with contextlib.redirect_stdout(None):
    %cd yolov5

# Esto es una prueba

In [33]:
btn_upload = widgets.FileUpload()
out_pl=widgets.Output()
btn_run = widgets.Button(description='Descripción')
lbl_pred = widgets.Label()
def on_click_descripcion(change):
    img=Image.open(io.BytesIO(btn_upload.data[-1]))
    name=list(btn_upload.value.keys())[-1]
    if " " in name:
        name="".join(name.split(" "))
        name=re.sub(r"[\(\)]",'',name)
    img_dir=os.path.join("../Imagenes_detect",name)
    img.save(img_dir)
    out_pl.clear_output()
    with out_pl: 
        display(img)
        words=results(name,"../Imagenes_detect")
btn_run.on_click(on_click_descripcion)
VBox([widgets.Label('Selecciona una ecografía'), 
      btn_upload, btn_run, out_pl])

benign(9).png


In [39]:
from platform import python_version
python_version()

'3.7.13'